# TODOs
- [ ] Runtimes with various input size and different optimization flags
> Create a timing infrastructure to get runtimes for a suitable set of input sizes (and inputs). Push sizes to the limit (i.e., until the execution takes minutes or even hours). Consider different compiler flags (but always with optimization).
- [ ] Cost Analysis (float ops, float comp, int ops, int comp)
> Cost analysis: Select a cost measure (usually flop count but may include div, sin, ...) and determine the cost. Briefly say how you did it (e.g., counted in code, had to instrument parts, ...).
- [ ] Performance Plot
> Create a performance plot. Put it as plot0 (or plot0a, plot0b, ...) into svn in the same directory as this document. As you perform optimizations later, include one line for each relevant optimization.
- [ ] Bottlenecks by profiling tool
> Identify performance bottlenecks in your code by profiling. This will likely involve instrumenting your code (standard profilers may not be fine grain enough). Focus on those parts when optimizing. The bottlenecks may change during optimization. Understand the cost and performance of the bottlenecks to set expectations right.

In [2]:
import re
import numpy as np    
import pandas as pd

#o0 = "./logo0.txt"
o1 = "./logo1.txt"
o2 = "./logo2.txt"
o3 = "./logo3.txt"

In [ ]:
with open(o1, 'r') as f:
    file = f.read()
res = re.findall("(.*)flops / cycles", file)
len(res)

In [3]:
def extract(log, metric):
    res = []
    with open(log, 'r') as f:
        file = f.read()
    try:
        if metric=='runtime(sec)':
            res = re.findall("(.*) seconds time elapsed", file)
        elif metric=='cc_int_ops':
            res = re.findall("cc_int_ops:(.*), cc_int_comp", file)
        elif metric=='cc_int_comp':
            res = re.findall("cc_int_comp:(.*), cc_fl_ops", file)
        elif metric=='cc_fl_ops':
            res = re.findall("cc_fl_ops(.*), cc_fl_comp", file)
        elif metric=='cc_fl_comp':
            res = re.findall("cc_fl_comp:(.*)", file)
        elif metric=='ll_int_ops':
            res = re.findall("ll_int_ops:(.*), ll_int_comp:", file)
        elif metric=='ll_int_comp':
            res = re.findall("ll_int_comp:(.*)", file)
        elif metric=='flops':
            res = re.findall("#fl ops: (.*)", file)
        elif metric=='flcomp':
            res = re.findall("#fl comparisons: (.*)", file)
        elif metric=='intops':
            res = re.findall("#int ops: (.*)", file)
        elif metric=='intcomp':
            res = re.findall("#int comparisons: (.*)", file)
        elif metric=='cycles':
            res = re.findall("#cycles: (.*)", file)
        elif metric=='flops/cycles':
            res = re.findall("(.*)flops / cycles", file)
        elif metric=='instructions':
            res = re.findall("(.*)   instructions:u", file)
        elif metric=='branches':
            res = re.findall("(.*)     branches:u", file)
        elif metric=='branch-misses':
            res = re.findall("branch-misses:u           #(.*) of all branches", file)
        elif metric=='page-faults':
            res = re.findall("(.*)  page-faults:u", file)
        elif metric=='cycles(perf)':
            res = re.findall("(.*)      cycles:u", file)
    except Exception:
        return 
    return res  

def init_table(vertice, edge):
    ve = []
    for v in vertice:
        for e in edge:
            if e < v*v:
                ve.append([v,e])
    return pd.DataFrame(ve, columns=['#V', '#E'])

def update_table(df, log, vertice, edge, data):
    for i, x in enumerate(vertice):
        for j, y in enumerate(edge):
            for l in label:
                data[l] = extract(log, l)
                if len(data[l])==len(df):
                    df[l] = data[l]
                else:
                    print(f"fail to add {l} to the table.")

In [4]:
from itertools import product

label=['runtime(sec)',
       'flops','flcomp','intops','intcomp', 'flops/cycles',
      'cc_int_ops', 'cc_int_comp', 'cc_fl_ops', 'cc_fl_comp', 'll_int_ops', 'll_int_comp',
      'instructions', 'cycles', 'cycles(perf)', 
      'branches', 'branch-misses', 'page-faults']

vertice = [10,100,1000,10000]
edge = [10,100,1000,10000,100000]

In [7]:
# results with flag -O3
data_o3 = dict()
res_o3 = init_table(vertice, edge)
update_table(res_o3, o3, vertice, edge, data_o3)
res_o3
res_o3.to_excel('result.xlsx', sheet_name='o3', index=False)

In [6]:
# results with no optimization
data_o1 = dict()
res_o1 = init_table(vertice, edge)
update_table(res_o1, o1, vertice, edge, data_o1)
res_o1
res_o1.to_excel('result.xlsx', sheet_name='o1', index=False)